In [154]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import ibm_boto3
from ibm_botocore.client import Config, ClientError
import time
import numpy as np
from io import BytesIO
import urllib.request as req
import wget
from PIL import Image
from datetime import datetime, date
import io as libio
import sched, time
from datetime import datetime, timedelta
from threading import Timer
import xml.dom.minidom
from xml.etree import ElementTree

In [59]:
cities_df = pd.read_csv('us_cities.csv')
cities_df

,City,State,Population,Latitude,Longitude,APICode
0,New York,New York,8398748,40.6635,-73.9387,40.6635%2C-73.9387
1,Los Angeles,California,3990456,34.0194,-118.41,34.0194%2C-118.41
2,Chicago,Illinois,2705994,41.8376,-87.6818,41.8376%2C-87.6818
3,Houston,Texas,2325502,29.7866,-95.3909,29.7866%2C-95.3909
4,Phoenix,Arizona,1660272,33.5722,-112.09,33.5722%2C-112.09
5,Philadelphia,Pennsylvania,1584138,40.0094,-75.1333,40.0094%2C-75.1333
6,San Antonio,Texas,1532233,29.4724,-98.5251,29.4724%2C-98.5251
7,San Diego,California,1425976,32.8153,-117.135,32.8153%2C-117.135
8,Dallas,Texas,1345047,32.7933,-96.7665,32.7933%2C-96.7665
9,San Jose,California,1030119,37.2967,-121.818,37.2967%2C-121.818


In [60]:
location = requests.get("http://dataservice.accuweather.com/locations/v1/cities/geoposition/search?apikey=9vvRWrL3ioBZFzTcuXg1QPZz0f1Xssu0&q=40.6635%2C73.9387")

In [61]:
location_df = json_normalize(location.json())

In [62]:
location_df

,AdministrativeArea.CountryID,AdministrativeArea.EnglishName,AdministrativeArea.EnglishType,AdministrativeArea.ID,AdministrativeArea.Level,AdministrativeArea.LocalizedName,AdministrativeArea.LocalizedType,Country.EnglishName,Country.ID,Country.LocalizedName,...,Region.ID,Region.LocalizedName,SupplementalAdminAreas,TimeZone.Code,TimeZone.GmtOffset,TimeZone.IsDaylightSaving,TimeZone.Name,TimeZone.NextOffsetChange,Type,Version
0,KG,Osh,Region,O,1,Osh,Region,Kyrgyzstan,KG,Kyrgyzstan,...,ASI,Asia,[],KGT,6.0,False,Asia/Bishkek,None,City,1


In [63]:
location_columns = list(location_df.columns) 

In [64]:
location_columns

['AdministrativeArea.CountryID',
 'AdministrativeArea.EnglishName',
 'AdministrativeArea.EnglishType',
 'AdministrativeArea.ID',
 'AdministrativeArea.Level',
 'AdministrativeArea.LocalizedName',
 'AdministrativeArea.LocalizedType',
 'Country.EnglishName',
 'Country.ID',
 'Country.LocalizedName',
 'DataSets',
 'EnglishName',
 'GeoPosition.Elevation.Imperial.Unit',
 'GeoPosition.Elevation.Imperial.UnitType',
 'GeoPosition.Elevation.Imperial.Value',
 'GeoPosition.Elevation.Metric.Unit',
 'GeoPosition.Elevation.Metric.UnitType',
 'GeoPosition.Elevation.Metric.Value',
 'GeoPosition.Latitude',
 'GeoPosition.Longitude',
 'IsAlias',
 'Key',
 'LocalizedName',
 'PrimaryPostalCode',
 'Rank',
 'Region.EnglishName',
 'Region.ID',
 'Region.LocalizedName',
 'SupplementalAdminAreas',
 'TimeZone.Code',
 'TimeZone.GmtOffset',
 'TimeZone.IsDaylightSaving',
 'TimeZone.Name',
 'TimeZone.NextOffsetChange',
 'Type',
 'Version']

In [65]:
for col in location_columns:
    cities_df[col] = ""

cities_df

,City,State,Population,Latitude,Longitude,APICode,AdministrativeArea.CountryID,AdministrativeArea.EnglishName,AdministrativeArea.EnglishType,AdministrativeArea.ID,...,Region.ID,Region.LocalizedName,SupplementalAdminAreas,TimeZone.Code,TimeZone.GmtOffset,TimeZone.IsDaylightSaving,TimeZone.Name,TimeZone.NextOffsetChange,Type,Version
0,New York,New York,8398748,40.6635,-73.9387,40.6635%2C-73.9387,,,,,...,,,,,,,,,,
1,Los Angeles,California,3990456,34.0194,-118.41,34.0194%2C-118.41,,,,,...,,,,,,,,,,
2,Chicago,Illinois,2705994,41.8376,-87.6818,41.8376%2C-87.6818,,,,,...,,,,,,,,,,
3,Houston,Texas,2325502,29.7866,-95.3909,29.7866%2C-95.3909,,,,,...,,,,,,,,,,
4,Phoenix,Arizona,1660272,33.5722,-112.09,33.5722%2C-112.09,,,,,...,,,,,,,,,,
5,Philadelphia,Pennsylvania,1584138,40.0094,-75.1333,40.0094%2C-75.1333,,,,,...,,,,,,,,,,
6,San Antonio,Texas,1532233,29.4724,-98.5251,29.4724%2C-98.5251,,,,,...,,,,,,,,,,
7,San Diego,California,1425976,32.8153,-117.135,32.8153%2C-117.135,,,,,...,,,,,,,,,,
8,Dallas,Texas,1345047,32.7933,-96.7665,32.7933%2C-96.7665,,,,,...,,,,,,,,,,
9,San Jose,California,1030119,37.2967,-121.818,37.2967%2C-121.818,,,,,...,,,,,,,,,,


In [66]:
for index, row in cities_df.iterrows():
    get_APICode = row['APICode']
    create_APIcall = "http://dataservice.accuweather.com/locations/v1/cities/geoposition/search?apikey=SnUPrdKxuZcUsSzXWDl5cae481d5hqNA&q=" + get_APICode
    location = requests.get(create_APIcall)
    location_df = json_normalize(location.json())
    for col in location_columns:
        if col in location_df.columns:
            cities_df.at[index,col] = location_df.at[0,col]

cities_df

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

,City,State,Population,Latitude,Longitude,APICode,AdministrativeArea.CountryID,AdministrativeArea.EnglishName,AdministrativeArea.EnglishType,AdministrativeArea.ID,...,Region.ID,Region.LocalizedName,SupplementalAdminAreas,TimeZone.Code,TimeZone.GmtOffset,TimeZone.IsDaylightSaving,TimeZone.Name,TimeZone.NextOffsetChange,Type,Version
0,New York,New York,8398748,40.6635,-73.9387,40.6635%2C-73.9387,US,New York,State,NY,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'Kings', 'Engli...",EST,-5,False,America/New_York,2020-03-08T07:00:00Z,City,1
1,Los Angeles,California,3990456,34.0194,-118.41,34.0194%2C-118.41,US,California,State,CA,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'Los Angeles', ...",PST,-8,False,America/Los_Angeles,2020-03-08T10:00:00Z,City,1
2,Chicago,Illinois,2705994,41.8376,-87.6818,41.8376%2C-87.6818,US,Illinois,State,IL,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'Cook', 'Englis...",CST,-6,False,America/Chicago,2020-03-08T08:00:00Z,City,1
3,Houston,Texas,2325502,29.7866,-95.3909,29.7866%2C-95.3909,US,Texas,State,TX,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'Harris', 'Engl...",CST,-6,False,America/Chicago,2020-03-08T08:00:00Z,City,1
4,Phoenix,Arizona,1660272,33.5722,-112.09,33.5722%2C-112.09,US,Arizona,State,AZ,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'Maricopa', 'En...",MST,-7,False,America/Phoenix,None,City,1
5,Philadelphia,Pennsylvania,1584138,40.0094,-75.1333,40.0094%2C-75.1333,US,Pennsylvania,State,PA,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'Philadelphia',...",EST,-5,False,America/New_York,2020-03-08T07:00:00Z,City,1
6,San Antonio,Texas,1532233,29.4724,-98.5251,29.4724%2C-98.5251,US,Texas,State,TX,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'Bexar', 'Engli...",CST,-6,False,America/Chicago,2020-03-08T08:00:00Z,City,1
7,San Diego,California,1425976,32.8153,-117.135,32.8153%2C-117.135,US,California,State,CA,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'San Diego', 'E...",PST,-8,False,America/Los_Angeles,2020-03-08T10:00:00Z,City,1
8,Dallas,Texas,1345047,32.7933,-96.7665,32.7933%2C-96.7665,US,Texas,State,TX,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'Dallas', 'Engl...",CST,-6,False,America/Chicago,2020-03-08T08:00:00Z,City,1
9,San Jose,California,1030119,37.2967,-121.818,37.2967%2C-121.818,US,California,State,CA,...,NAM,North America,"[{'Level': 2, 'LocalizedName': 'Santa Clara', ...",PST,-8,False,America/Los_Angeles,2020-03-08T10:00:00Z,City,1


In [133]:
cities_df.to_csv('us_cities_allinfo.csv')


PermissionError: [Errno 13] Permission denied: 'us_cities_allinfo.csv'

In [134]:
cities_df = pd.read_csv('us_cities_allinfo.csv')

In [147]:
data_col = ['key','city', 'state', 'lat', 'lon', 'time', 'timeZone', 'obsDaylight', 'currentGmtOffset', 'timeZoneAbbreviation', \
       'observationtime', 'pressurestate', 'temperature', 'realfeel', 'humidity', 'weathertext', 'weathericon', 'windgusts', \
       'windspeed', 'winddirection', 'visibility', 'precip', 'uvindex', 'dewpoint', 'cloudcover']

for index, row in cities_df.iterrows():
    get_locationKey = row['Key']
    
    current = requests.get("http://samsungmobile.accu-weather.com/widget/samsungmobile/weather-data.asp?metric=1&location=cityId%3A"+ str(get_locationKey))
    root = ElementTree.fromstring(current.content)
    
    conditions = [[get_locationKey ,root[1][0].text, root[1][1].text, root[1][2].text, root[1][3].text, root[1][4].text, root[1][5].text, \
              root[1][6].text, root[1][7].text, root[1][8].text, root[3][1].text, root[3][2].text, root[3][3].text, \
              root[3][4].text, root[3][5].text, root[3][6].text, root[3][7].text, root[3][8].text, root[3][9].text, \
              root[3][10].text, root[3][11].text, root[3][12].text, root[3][13].text, root[3][14].text, root[3][15].text]]
    
    current_df = pd.DataFrame(conditions, columns=data_col)
    weather_24_us = pd.concat([weather_24_us, current_df])

In [148]:
weather_24_us

,key,city,state,lat,lon,time,timeZone,obsDaylight,currentGmtOffset,timeZoneAbbreviation,...,weathertext,weathericon,windgusts,windspeed,winddirection,visibility,precip,uvindex,dewpoint,cloudcover
0,2102350,East Flatbush,New York,40.65372,-73.93042,12:18,-5,\n\t0\n\t,-5,EST,...,Mostly cloudy,06,14,10,NW,16,0.0,Low,-4,76%
0,2178430,Palms,California,34.02279,-118.4056,08:44,-8,\n\t0\n\t,-8,PST,...,Sunny,01,0,0,N,16,0.0,Low,4,0%
0,2094443,Lower West Side,Illinois,41.85421,-87.66561,10:44,-6,\n\t0\n\t,-6,CST,...,Mostly cloudy,06,10,6,SW,14,0.0,Low,-2,87%
0,351197,Houston,Texas,29.76329,-95.36328,11:00,-6,\n\t0\n\t,-6,CST,...,Sunny,01,22,11,N,16,0.0,Moderate,6,0%
0,346935,Phoenix,Arizona,33.44838,-112.0741,09:11,-7,\n\t0\n\t,-7,MST,...,Partly sunny,03,0,0,N,16,0.0,Low,7,31%
0,2155989,Hunting Park,Pennsylvania,40.01651,-75.14379,11:44,-5,\n\t0\n\t,-5,EST,...,Sunny,01,9,6,SSE,16,0.0,Low,-2,10%
0,351198,San Antonio,Texas,29.42413,-98.49364,10:45,-6,\n\t0\n\t,-6,CST,...,Mostly sunny,02,7,4,NNE,16,0.0,Moderate,7,11%
0,2628148,Kearny Mesa,California,32.82806,-117.1429,08:44,-8,\n\t0\n\t,-8,PST,...,Partly sunny,03,5,5,SSE,16,0.0,Low,8,31%
0,351194,Dallas,Texas,32.78306,-96.80667,10:28,-6,\n\t0\n\t,-6,CST,...,Cloudy,07,9,9,WNW,16,0.0,Low,2,91%
0,347630,San Jose,California,37.33939,-121.895,08:48,-8,\n\t0\n\t,-8,PST,...,Mostly sunny,02,0,0,N,16,0.0,Low,6,30%


In [145]:
weather_24_us.drop(weather_24_us.index, inplace=True)

In [174]:
dayandtime = []

today = datetime.today()
for hr in range(24):
    yesterday = today - timedelta(days = 1)
    yesterday = yesterday.replace(hour=0, minute=0)
    dayandtime.append(yesterday.strftime('%m%d%H'))
    
for hr in range(24):
    today = today.replace(hour=0, minute=0)
    today = today + timedelta(hours=hr)
    dayandtime.append(today.strftime('%m%d%H'))

print(dayandtime)


['112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112200', '112300', '112301', '112302', '112303', '112304', '112305', '112306', '112307', '112308', '112309', '112310', '112311', '112312', '112313', '112314', '112315', '112316', '112317', '112318', '112319', '112320', '112321', '112322', '112323']
